In [1]:
pip install transformers

     |████████████████████████████████| 3.5 MB 11.8 MB/s 
     |████████████████████████████████| 6.8 MB 43.0 MB/s 
     |████████████████████████████████| 596 kB 27.4 MB/s 
     |████████████████████████████████| 67 kB 3.4 MB/s 
     |████████████████████████████████| 895 kB 45.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install pytorch_transformers

     |████████████████████████████████| 176 kB 16.9 MB/s 
     |████████████████████████████████| 131 kB 13.8 MB/s 
     |████████████████████████████████| 1.2 MB 40.6 MB/s 
     |████████████████████████████████| 8.5 MB 53.8 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 138 kB 70.3 MB/s 
     |████████████████████████████████| 127 kB 45.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import torch
from pytorch_transformers import BertTokenizer
from pytorch_transformers import BertModel
## Load pretrained model/tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True)

100%|██████████| 440473133/440473133 [00:24<00:00, 18124709.28B/s]


In [4]:
def final(text):
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])

  with torch.no_grad():
    outputs = model(tokens_tensor)
    last_hidden_state = outputs[0]
    word_embed_1 = last_hidden_state
# Evaluating the model will return a different number of objects               based on how it's  configured in the `from_pretrained` call earlier. In this case, becase we set `output_hidden_states = True`, the third item will be the hidden states from all layers. See the documentation for more details:https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
    word_embed_6 = torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1)

    sentence_embedding = torch.mean(word_embed_6, dim=1)

  return sentence_embedding


In [5]:
# Define an input text
text = "Here is the sentence I want embeddings for."
# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"
# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)
# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
here          2,182
is            2,003
the           1,996
sentence      6,251
i             1,045
want          2,215
em            7,861
##bed         8,270
##ding        4,667
##s           2,015
for           2,005
.             1,012
[SEP]           102


In [ ]:
import torch
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
# Put the model in "evaluation" mode,meaning feed-forward operation.
model.eval()

In [7]:
#Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
with torch.no_grad():
    outputs = model(tokens_tensor)
# can use last hidden state as word embeddings
    last_hidden_state = outputs[0]
    word_embed_1 = last_hidden_state
# Evaluating the model will return a different number of objects               based on how it's  configured in the `from_pretrained` call earlier. In this case, becase we set `output_hidden_states = True`, the third item will be the hidden states from all layers. See the documentation for more details:https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
# initial embeddings can be taken from 0th layer of hidden states
    word_embed_2 = hidden_states[0]
# sum of all hidden states
    word_embed_3 = torch.stack(hidden_states).sum(0)
# sum of second to last layer
    word_embed_4 = torch.stack(hidden_states[2:]).sum(0)
# sum of last four layer
    word_embed_5 = torch.stack(hidden_states[-4:]).sum(0)
# concatenate last four layers
    word_embed_6 = torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1)

In [8]:
word_embed_6.shape

torch.Size([1, 14, 3072])

In [9]:
sentence_embedding = torch.mean(word_embed_6, dim=1)

In [10]:
sentence_embedding.shape

torch.Size([1, 3072])

In [11]:
sentence_embedding

tensor([[ 0.1472, -0.1808,  0.1119,  ...,  0.1263, -0.0970,  0.0809]])

In [12]:
texts = ['Here is the sentence I want embeddings for.', 'Hi Its been a great day but i am not sure what the next']

In [13]:
final(texts[0])

tensor([[ 0.1472, -0.1808,  0.1119,  ...,  0.1263, -0.0970,  0.0809]])

In [14]:
from google.colab import files
uploaded = files.upload()

Saving Arg-Rank.csv to Arg-Rank.csv


In [15]:
import numpy as np
import pandas as pd

In [16]:
df = pd.read_csv('Arg-Rank.csv')
df

,id,rank,argument
0,22,0.625,90% of traffic accidents are due (at least par...
1,43,0.8,A large part of the inefficiencies of vehicles...
2,65,0.25,AI is cleverer than human beings. AI can do be...
3,66,0.230769231,AI is more consistent when it comes to doing r...
4,80,0.642857143,According to the United States Department of T...
...,...,...,...
5312,5252,0.235294118,it is moral to eat meat as animals are less de...
5313,5277,0.736842105,meat is a necessary component of a human's die...
5314,5278,0.333333333,meat substitutes like vitamins and soy product...
5315,5379,0.181818182,some people will loose the joy in eating. a jo...


In [17]:
len(df)

5317

In [18]:
len(list(set(df['id'])))

5299

In [19]:
df.drop_duplicates(keep=False,inplace=True)

In [20]:
df

,id,rank,argument
0,22,0.625,90% of traffic accidents are due (at least par...
1,43,0.8,A large part of the inefficiencies of vehicles...
2,65,0.25,AI is cleverer than human beings. AI can do be...
3,66,0.230769231,AI is more consistent when it comes to doing r...
4,80,0.642857143,According to the United States Department of T...
...,...,...,...
5312,5252,0.235294118,it is moral to eat meat as animals are less de...
5313,5277,0.736842105,meat is a necessary component of a human's die...
5314,5278,0.333333333,meat substitutes like vitamins and soy product...
5315,5379,0.181818182,some people will loose the joy in eating. a jo...


In [21]:
arg = df['argument']

In [22]:
scores = df['rank']

In [23]:
df.reset_index(inplace=True)

In [24]:
df

,index,id,rank,argument
0,0,22,0.625,90% of traffic accidents are due (at least par...
1,1,43,0.8,A large part of the inefficiencies of vehicles...
2,2,65,0.25,AI is cleverer than human beings. AI can do be...
3,3,66,0.230769231,AI is more consistent when it comes to doing r...
4,4,80,0.642857143,According to the United States Department of T...
...,...,...,...,...
5293,5312,5252,0.235294118,it is moral to eat meat as animals are less de...
5294,5313,5277,0.736842105,meat is a necessary component of a human's die...
5295,5314,5278,0.333333333,meat substitutes like vitamins and soy product...
5296,5315,5379,0.181818182,some people will loose the joy in eating. a jo...


In [25]:
argm = []
for i in range(len(df)):
  argm.append(final(df['argument'][i]))

In [26]:
len(argm)

5298

In [27]:
y = scores
y = [float(x) for x in y ]
y = [np.array(x) for x in y ]

In [28]:
len(y)

5298

In [29]:
X = argm

In [30]:
X[1]

tensor([[-0.2307, -0.0077,  0.3101,  ..., -0.5030,  0.0746,  0.2134]])

In [31]:
X = [X[i][0] for i in range(len(X))]

In [32]:
X[0]

tensor([ 0.1228,  0.0097,  0.3688,  ..., -0.2915,  0.1532,  0.0523])

In [33]:
X = [t.numpy() for t in X]

In [34]:
X[0]

array([ 0.12275673,  0.00967564,  0.36876333, ..., -0.29147962,
        0.15319403,  0.05232434], dtype=float32)

In [35]:
X = np.array(X)

In [36]:
X

array([[ 0.12275673,  0.00967564,  0.36876333, ..., -0.29147962,
         0.15319403,  0.05232434],
       [-0.23070434, -0.00772764,  0.31014228, ..., -0.50298125,
         0.07459059,  0.21336383],
       [ 0.14991984,  0.100587  ,  0.0375925 , ..., -0.32052281,
         0.27706704,  0.10051783],
       ...,
       [ 0.34920967,  0.38980222, -0.2425646 , ..., -0.39427277,
        -0.14810982, -0.2481087 ],
       [ 0.07964652,  0.03577435,  0.22772433, ..., -0.43527249,
         0.08850057, -0.16459489],
       [ 0.40399274,  0.1194367 ,  0.15534543, ..., -0.33160922,
         0.41247615, -0.30330965]], dtype=float32)

In [37]:
y = np.array(y)

In [38]:
y

array([0.625     , 0.8       , 0.25      , ..., 0.33333333, 0.18181818,
       0.58823529])

In [39]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
# Keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
#from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

In [40]:
from tensorflow.keras.layers import Dense

In [50]:
model = Sequential()
model.add(Dense(300, input_shape = (3072,), activation = "relu"))
#model.add(Dropout(0.2))
model.add(Dense(1, activation = "sigmoid"))
model.compile(Adam(learning_rate = 1e-7), "mean_squared_error", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 300)               921900    
                                                                 
 dense_3 (Dense)             (None, 1)                 301       
                                                                 
Total params: 922,201
Trainable params: 922,201
Non-trainable params: 0
_________________________________________________________________


In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [52]:
model.fit(x_train, y_train, verbose=1, epochs=200)

Epoch 1/200
133/133 [==============================] - 2s 14ms/step - loss: 0.0652 - accuracy: 0.0109
Epoch 2/200
133/133 [==============================] - 2s 13ms/step - loss: 0.0602 - accuracy: 0.0137
Epoch 3/200
133/133 [==============================] - 2s 12ms/step - loss: 0.0561 - accuracy: 0.0165
Epoch 4/200
133/133 [==============================] - 1s 9ms/step - loss: 0.0527 - accuracy: 0.0191
Epoch 5/200
133/133 [==============================] - 1s 9ms/step - loss: 0.0501 - accuracy: 0.0201
Epoch 6/200
133/133 [==============================] - 1s 9ms/step - loss: 0.0480 - accuracy: 0.0238
Epoch 7/200
133/133 [==============================] - 1s 9ms/step - loss: 0.0464 - accuracy: 0.0267
Epoch 8/200
133/133 [==============================] - 1s 9ms/step - loss: 0.0452 - accuracy: 0.0278
Epoch 9/200
133/133 [==============================] - 1s 9ms/step - loss: 0.0443 - accuracy: 0.0281
Epoch 10/200
133/133 [==============================] - 1s 9ms/step - loss: 0.0436 - acc

In [53]:
predict_x=model.predict(x_test) 

In [54]:
predict_x[0]

array([0.5062029], dtype=float32)

In [55]:
p = list(predict_x)

In [56]:
p = [ p[i][0] for i in range(len(p)) ]

In [57]:
y_test[0],p[0]

(0.333333333, 0.5062029)

In [58]:
# calculate the Pearson's correlation between two variables
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
from scipy.stats import spearmanr
# seed random number generator
seed(1)
# prepare data

# calculate Pearson's correlation
corr, _ = pearsonr(y_test,p)
print('Pearsons correlation: %.3f' % corr,'\n')
corr2, _ = spearmanr(y_test,p)
print('Spearmans correlation: %.3f' % corr2)

Pearsons correlation: 0.408 

Spearmans correlation: 0.394
